# Introduction to iFEM

## Geometry of the problem : 

A beam from point A to B simply supported with length L, Youngs modulus E and Inertia I :
                                                                              
         A O___________________________________________________O   B     
          /\                      L                           /\         
         /  \                                                /  \        

nel, the number of elements for spatial discretization

## Convenient imports and tools

In [1]:
using Muscade, StaticArrays, GLMakie, CSV, DataFrames, Interpolations
using Muscade.Toolbox
include("save_results.jl")

save_metadata (generic function with 1 method)

## Model creation

In [2]:
# R   = 0.0;          # Radius of the bend [m]
EI₂ = 1e5;          # Bending stiffness [Nm²]
EI₃ = 1e5;          # Bending stiffness [Nm²]
EA  = 1e6;          # Axial stiffness [N]
GJ  = 1e6;          # Torsional stiffness [Nm²]
L   = 10.;          # Length of the beam [m]
μ   = 1.;           # Linear mass along main axis [kg/m]
ι₁  = 1.;           # Mass moment of inertia around main axis [kg·m³]

nel         = 50
nnodes      = nel+1
nodeCoord   = hcat( -5. .+ ((1:nnodes).-1)/(nnodes-1)*L,
                     0  .+ zeros(Float64, nnodes, 1),
                     0  .+ zeros(Float64, nnodes, 1))
mat         = BeamCrossSection(EA=EA,EI₂=EI₂,EI₃=EI₃,GJ=GJ,μ=1.,ι₁=1.)

BeamCrossSection(1.0e6, 100000.0, 100000.0, 1.0e6, 1.0, 1.0)

In [3]:
function createSimplySupportedBeam(name::Symbol; bPlanar=false)
    model       = Model(name)
    nodid       = addnode!(model, nodeCoord)
    mesh        = hcat(nodid[1:nnodes-1],nodid[2:nnodes])
    eleid       = addelement!(model, EulerBeam3D, mesh;mat=mat, orient2=SVector(0.,1.,0.))
    [addelement!(model,Hold,[nodid[1]]  ;field) for field∈[:t1,:t2,:t3,:r1]];   # Support at one end
    [addelement!(model,Hold,[nodid[nnodes]]  ;field) for field∈[:t1, :t2,:t3,:r1]];      # Support at the other end
    if bPlanar 
        [[addelement!(model,Hold,[nodid[i]] ;field) for field∈[:t3]] for i in 2:nnodes-1] 
    end
    return model, nodid, nnodes, eleid
end

createSimplySupportedBeam (generic function with 1 method)

In [4]:

function createDampedSimplySupportedBeam(name::Symbol, β::Float64; bPlanar=false)
    model       = Model(name)
    nodid       = addnode!(model, nodeCoord)
    mesh        = hcat(nodid[1:nnodes-1],nodid[2:nnodes])
    damped_mat         = DampedBeamCrossSection(EA=EA,EI₂=EI₂,EI₃=EI₃,GJ=GJ,μ=1.,ι₁=1.,β=β)
    eleid       = addelement!(model, EulerBeam3D, mesh;mat=damped_mat, orient2=SVector(0.,1.,0.))
    [addelement!(model,Hold,[nodid[1]]  ;field) for field∈[:t1,:t2,:t3,:r1]];   # Support at one end
    [addelement!(model,Hold,[nodid[nnodes]]  ;field) for field∈[:t1, :t2,:t3,:r1]];      # Support at the other end
    if bPlanar 
        [[addelement!(model,Hold,[nodid[i]] ;field) for field∈[:t3]] for i in 2:nnodes-1] 
    end
    return model, nodid, nnodes
end

createDampedSimplySupportedBeam (generic function with 1 method)

## Load scenarii
### Impulse load

In [ ]:
function impulse_load(A, t, t₀; Δt = 0.05)
    abs(t-t₀)<Δt ? impulse_load = A : impulse_load = 0.
end

function impulse_load(A, t, t₀, T; Δt = 0.05)
    abs(t%T-t₀)<Δt ? impulse_load = A : impulse_load = 0.
end

### Sinusoidal load

In [ ]:
function sinus_load(A, t, T, ϕ)
    sinus_load = A * sin(2*π*t/T + ϕ)
end

function distributed_sinus_load(q, t, T_max, L, x)
    distributed_sinus_load = q*(t/T_max)*sin(2*π*x/L)
end

### Ramp load

In [ ]:
function ramp_load(A, T, t)
    t₁ = T[1]
    t₂ = T[2]
    t < t₁ ? ramp_load = 0.0 :
    t < t₂ ? ramp_load = A*(t-t₁)/(t₂-t₁) :
    ramp_load = A
end

## Static analysis

In [5]:
model_stat, nodid, nnodes   = createSimplySupportedBeam(:StaticAnalysis)

# Static_Loads                = [A for A in -50000.:-50000.:-150000.]
A = -1e3
@functor with(A) constant_load(t) = t >= 100 ? A : A/100*(t+1)
# @functor with(A) constant_load(t) = A
addelement!(model_stat,DofLoad,[nodid[floor(Int,nnodes/2)]];field=:t3,value=constant_load )

initialstate                = initialize!(model_stat);

loadSteps                   = [i for i in 1.:10.:500.];
nLoadSteps                  = length(loadSteps)
state                       = solve(SweepX{0};initialstate=initialstate,time=loadSteps,verbose=true,maxΔx=1e-8, maxiter = 50);




Muscade: SweepX{0} solver

    step   1 converged in   3 iterations. |Δx|=8.3e-10 |Lλ|=9.2e-07
    step   2 converged in   4 iterations. |Δx|=3.7e-11 |Lλ|=4.3e-09
    step   1 converged in   3 iterations. |Δx|=8.3e-10 |Lλ|=9.2e-07
    step   2 converged in   4 iterations. |Δx|=3.7e-11 |Lλ|=4.3e-09
    step   3 converged in   4 iterations. |Δx|=1.4e-10 |Lλ|=9.5e-09
    step   4 converged in   4 iterations. |Δx|=4.4e-10 |Lλ|=1.2e-08
    step   5 converged in   4 iterations. |Δx|=7.6e-10 |Lλ|=2.0e-08
    step   3 converged in   4 iterations. |Δx|=1.4e-10 |Lλ|=9.5e-09
    step   4 converged in   4 iterations. |Δx|=4.4e-10 |Lλ|=1.2e-08
    step   5 converged in   4 iterations. |Δx|=7.6e-10 |Lλ|=2.0e-08
    step   6 converged in   4 iterations. |Δx|=1.0e-09 |Lλ|=2.1e-08
    step   7 converged in   4 iterations. |Δx|=1.3e-09 |Lλ|=2.2e-08
    step   8 converged in   4 iterations. |Δx|=1.4e-09 |Lλ|=3.1e-08
    step   6 converged in   4 iterations. |Δx|=1.0e-09 |Lλ|=2.1e-08
    step   7 conve

In [6]:
x_ = [getdof(state[idxLoad];field=:t1,nodID=nodid[1:nnodes]) for idxLoad ∈ 1:nLoadSteps]
y_ = [getdof(state[idxLoad];field=:t2,nodID=nodid[1:nnodes]) for idxLoad ∈ 1:nLoadSteps]
z_ = [getdof(state[idxLoad];field=:t3,nodID=nodid[1:nnodes]) for idxLoad ∈ 1:nLoadSteps]
r_ = [getdof(state[idxLoad];field=:r3,nodID=nodid[1:nnodes]) for idxLoad ∈ 1:nLoadSteps]

fig     = Figure(size = (1000,1000))
ax      = Axis3(fig[1,1],xlabel="x [m]", ylabel="y [m]", zlabel="z [m]",aspect=:equal, title = "Static analysis results")
clr     = [:black,:blue,:green,:red]
for idxLoad ∈ 1:nLoadSteps
    draw!(ax,state[idxLoad];EulerBeam3D=(;nseg=10, line_color = clr[idxLoad%4+1]))
end
xlims!(ax, -5,5); ylims!(ax, -5,5); zlims!(ax, -5,5);
fig

In [7]:
timeseries = Dict(
    "X" => x_,
    "Y" => y_,
    "Z" => z_,
    "R" => r_
)
save_timeseries_csv("results_combined"; metadata = mat, comps=timeseries, time=loadSteps)

true

## Eigenvalue analysis

In [ ]:
model_eig, nodid, nnodes   = createSimplySupportedBeam(:EigAnalysis; bPlanar = true)
[addelement!(model_eig,DofLoad,[nodid[nodeidx]];field=:t2,value=t-> sinus_load(200., t, 10., 0.)) for nodeidx=1:nnodes];

initialstate    = initialize!(model_eig);
state_eig       = solve(SweepX{0};initialstate,time=[0.]);
nmod            = 15
#res             = solve(EigX{ℝ};state=state_eig[1],nmod); Assembly has some issue at this date, will have to look at it a bit more at some point


## Dynamic analysis with various load scenarii
### Sinusoidal nodal load

In [ ]:
model_dyn, nodid, nnodes   = createSimplySupportedBeam(:DynAnalysis_sin)

A, Tp, ϕ = 1e4, 10., 0.
@functor with(A, T, ϕ) sin_load(t) = t == 0.15 ? A * sin(2*π*t/Tp + ϕ) : 0.0
addelement!(model_dyn,DofLoad,[nodid[floor(Int,nnodes/2)]];field=:t2,value= t -> sin_load )

initialstate                = initialize!(model_dyn; time = 0.);
Tsin                        = 1.:0.05:10.
nLoadSteps                  = length(Tsin)
state                       = solve(SweepX{2};initialstate,time=Tsin,verbose=true,maxΔx=1e-8, maxiter = 80);

In [ ]:
x_sin = [[getdof(state[idxLoad];field=:t1,nodID=[nodid[node]]) for idxLoad ∈ 1:nLoadSteps] for node in 1:nnodes]
y_sin = [[getdof(state[idxLoad];field=:t2,nodID=[nodid[node]]) for idxLoad ∈ 1:nLoadSteps] for node in 1:nnodes]
z_sin = [[getdof(state[idxLoad];field=:t3,nodID=[nodid[node]]) for idxLoad ∈ 1:nLoadSteps] for node in 1:nnodes]
r3_sin = [[getdof(state[idxLoad];field=:r3,nodID=[nodid[node]]) for idxLoad ∈ 1:nLoadSteps] for node in 1:nnodes]

figure     = Figure(size = (1000,1000))
ax      = Axis3(figure[1,1],xlabel="x [m]", ylabel="y [m]", zlabel="z [m]",aspect=:equal)
for to_draw in 1:10:nLoadSteps
    draw!(ax,state[to_draw];EulerBeam3D=(;nseg=20,  line_color= RGBf(1.0, to_draw/nLoadSteps, 0.)))
end


In [ ]:
req = @request gp(x∂X₀)
loads = getresult(statew[1500],req,[eleid[1]])
print(loads)

In [ ]:

figure     = Figure(size = (1000,1000))
ax      = Axis3(figure[1,1],xlabel="x [m]", ylabel="y [m]", zlabel="z [m]",aspect=:equal)
rng = 50:5:90 # 1:100:nLoadSteps

for to_draw in rng
    draw!(ax,statew[to_draw];EulerBeam3D=(;nseg=20,  line_color= RGBf((to_draw-rng[1])/length(rng)*0.5, 0.99.-(to_draw-rng[1])/length(rng)*0.5, 0.)))
end
display(figure)
figure

In [ ]:

fig_zt = Figure(size = (1000,1000))
x_ = [getdof(statew[idxLoad];field=:t1,nodID=[nodid[21]]) for idxLoad ∈ 1:nLoadSteps]
y_ = [getdof(statew[idxLoad];field=:t2,nodID=[nodid[10]]) for idxLoad ∈ 1:nLoadSteps]
z_ = [getdof(statew[idxLoad];field=:t3,nodID=[nodid[20]]) for idxLoad ∈ 1:nLoadSteps]
z_m = [getdof(statew[idxLoad];field=:t3,nodID=[nodid[10]]) for idxLoad ∈ 1:nLoadSteps]
ax = Axis(fig_zt[1, 1], xlabel="Time, t [s]", ylabel="Displacement in the y-direction [m]", title = "Direct solution to impulse load at t = 0.15s, load distributed on all nodes, No Damping")
lines!(ax, T, vcat(z_...); label="Node 20")
lines!(ax, T, vcat(z_m...); label="Node 10")
axislegend()


save("no_damping.png",fig_zt)


Local initial load

In [ ]:
model_dyn, nodid, nnodes   = createSimplySupportedBeam(:DynAnalysis_impulselocal)

A = 1000

addelement!(model_dyn,DofLoad,[nodid[floor(Int, nnodes/2)]];field=:t2,value= t -> t == 0.1 ? A : 0.0)

initialstate                = initialize!(model_dyn; time = 0.);

loadSteps                   = [i for i in 0.1:0.005:10.];
nLoadSteps                  = length(loadSteps)
statew                       = solve(SweepX{2};initialstate,time=loadSteps,verbose=true,maxΔx=1e-6, maxiter = 80);

In [ ]:
fig_zt = Figure(size = (1000,1000))
x_ = [getdof(statew[idxLoad];field=:t1,nodID=[nodid[6]]) for idxLoad ∈ 1:nLoadSteps]
y_ = [getdof(statew[idxLoad];field=:t2,nodID=[nodid[6]]) for idxLoad ∈ 1:nLoadSteps]
z_ = [getdof(statew[idxLoad];field=:t3,nodID=[nodid[6]]) for idxLoad ∈ 1:nLoadSteps]
Axis(fig_zt[1, 1])
scatter!(vcat(y_...))

fig_zt

In [ ]:
figure     = Figure(size = (1000,1000))
ax      = Axis3(figure[1,1],xlabel="x [m]", ylabel="y [m]", zlabel="z [m]",aspect=:equal)
for to_draw in [10,11, 12, 13, 14]
    draw!(ax,statew[to_draw];EulerBeam3D=(;nseg=20,  line_color= RGBf(to_draw/1500, 0., 0.)))
end
display(figure)
figure

## Inverse Crime

### Static inverse crime

#### Load datas

In [8]:
data = load_timeseries_csv("results_combined")
idxLoad = 50
x_restored = data.series["X"][:, idxLoad]
y_restored = data.series["Y"][:, idxLoad]
z_restored = data.series["Z"][:, idxLoad]
r3_restored = data.series["R"][:, idxLoad]
time = data.time;
println(size(x_restored))

(51,)


#### Create model and solve

In [ ]:
# inv_model = Model(:inverse_static)
# nodid2      = addnode!(inv_model, nodeCoord2)
# nnodes = size(nodid2, 1)
# mesh        = hcat(nodid2[1:nnodes-1],nodid2[2:nnodes])
# eleid       = addelement!(inv_model, EulerBeam3D, mesh;mat=mat, orient2=SVector(0.,1.,0.))
# [addelement!(inv_model,Hold,[nodid2[1]]  ;field) for field∈[:t1,:t2,:t3,:r1]];   # Support at one end
# [addelement!(inv_model,Hold,[nodid2[nnodes]]  ;field) for field∈[:t1, :t2,:t3,:r1]];      # Support at the other end

inv_model, nodid, nnodes, eleid   = createSimplySupportedBeam(:inverse_static)

# Helper: accept meas as either a scalar or a function of time
get_meas(meas, t) = isa(meas, Function) ? meas(t) : meas

A_estimated = -1e3
# Define cost functors with signature (x, t, meas) to match Muscade expectations
@functor with() costX(x, t, meas) = 1 * (get_meas(meas, t) - x)^2
@functor with() costXother(x, t, meas) = 1 * (get_meas(meas, t) - x)^2
@functor with(A_estimated) costU(u, t) = 1*(A_estimated-u)^2
@functor with() costUother(u, t) = 1*u^2

node_number = 10
e5 = [addelement!(inv_model,SingleDofCost,[nodid[node]];class=:X,field=:t1, cost= costXother, costargs=(meas = x_restored[node],) ) for node in 1:nnodes]
e6 = [addelement!(inv_model,SingleDofCost,[nodid[node]];class=:X,field=:t2, cost= costX,      costargs=(meas = y_restored[node],) ) for node in 1:nnodes]
e7 = [addelement!(inv_model,SingleDofCost,[nodid[node]];class=:X,field=:t3, cost= costXother, costargs=(meas = z_restored[node],) ) for node in 1:nnodes]
e7 = [addelement!(inv_model,SingleDofCost,[nodid[node]];class=:X,field=:r3, cost= costXother, costargs=(meas = r3_restored[node],) ) for node in 1:nnodes]
e2 = [addelement!(inv_model,SingleUdof,[nodid[node]]; Xfield=:t3, Ufield=:t3, cost=costUother ) for node in 1:nnodes-1];
e2 = [addelement!(inv_model,SingleUdof,[nodid[node]]; Xfield=:t2, Ufield=:t2, cost=costUother ) for node in 1:nnodes-1 if node != node_number];
e3 = [addelement!(inv_model,SingleUdof,[nodid[node_number]]; Xfield=:t2, Ufield=:t2, cost=costU )];
e4 = [addelement!(inv_model,SingleUdof,[nodid[node]]; Xfield=:t1, Ufield=:t1, cost=costUother ) for node in 1:nnodes-1];


# Muscade.describe(inv_model,:dof)
# Muscade.describe(inv_model,:doftyp)
# Muscade.describe(inv_model,:eletyp)

In [ ]:
InvSolver        = DirectXUA{0,0,0}   # Solver for the inverse analysis
maxiter     = 50
maxΔx       = 1e-6
maxΔu       = 1e-6
maxΔa       = 1e-6
maxΔλ       = Inf

initialstate = initialize!(inv_model);
# Use an AbstractRange wrapped in a Vector as required by the solver
inv_timeSteps = 500.0:-10.0:500.
nSteps = length(inv_timeSteps)

stateXUA = solve(InvSolver;
    initialstate = [initialstate],
    time = [inv_timeSteps],
    verbose = true,
    maxiter = maxiter,
    maxΔx = maxΔx,
    maxΔλ = maxΔλ,
    maxΔu = maxΔu,
    maxΔa = maxΔa,
);


### Extracting results

In [ ]:
idx = 1

# displacements
println("\n== Displacements :")
x_ = [getdof(stateXUA[idx];field=:t1,nodID=nodid[1:nnodes]) for idx ∈ 1:nSteps]
y_ = [getdof(stateXUA[idx];field=:t2,nodID=nodid[1:nnodes]) for idx ∈ 1:nSteps]
z_ = [getdof(stateXUA[idx];field=:t3,nodID=nodid[1:nnodes]) for idx ∈ 1:nSteps]
r_ = [getdof(stateXUA[idx];field=:r3,nodID=nodid[1:nnodes]) for idx ∈ 1:nSteps];

println(x_)
println(y_)
println(z_)
println(r_)

# Axial force
println("\n== Axial force :")
req = @request gp(resultants(fᵢ))
out = getresult(stateXUA[idx],req,eleid)
Fgp1_ = [ out[idxEl].gp[1][:resultants][:fᵢ] for idxEl ∈ 1:nel]
Fgp2_ = [ out[idxEl].gp[2][:resultants][:fᵢ] for idxEl ∈ 1:nel]
Fgp3_ = [ out[idxEl].gp[3][:resultants][:fᵢ] for idxEl ∈ 1:nel]
Fgp4_ = [ out[idxEl].gp[4][:resultants][:fᵢ] for idxEl ∈ 1:nel];

println(Fgp1_)
println(Fgp2_)
println(Fgp3_)
println(Fgp4_)

# Bending moments
println("\n== Bending moment :")
req = @request gp(resultants(mᵢ))
out = getresult(stateXUA[idx],req,eleid)
Mgp1_ = [ out[idxEl].gp[1][:resultants][:mᵢ] for idxEl ∈ 1:nel]
Mgp2_ = [ out[idxEl].gp[2][:resultants][:mᵢ] for idxEl ∈ 1:nel]
Mgp3_ = [ out[idxEl].gp[3][:resultants][:mᵢ] for idxEl ∈ 1:nel]
Mgp4_ = [ out[idxEl].gp[4][:resultants][:mᵢ] for idxEl ∈ 1:nel];

println(Mgp1_)
println(Mgp2_)
println(Mgp3_)
println(Mgp4_)

In [ ]:
# Plot axial force resultants (Fgp1_ from previous extraction cell)
using GLMakie
fig = Figure()
ax = Axis(fig[1,1], xlabel="Element index", ylabel="Axial force fᵢ", title="Axial force along elements (gp1)")
lines!(ax, 1:length(Fgp1_), Fgp1_; linewidth=2)
scatter!(ax, 1:length(Fgp1_), Fgp1_; markersize=5)
fig

In [ ]:
# Plot bending moments - multiple approaches
using GLMakie, LinearAlgebra

# Approach 1: Magnitude of moment vectors at Gauss point 1
Mgp1_magnitude = [norm(m) for m in Mgp1_]

fig = Figure(size=(1200, 800))

# Subplot 1: Magnitude of M at gp1
ax1 = Axis(fig[1,1], xlabel="Element index", ylabel="Magnitude |mᵢ|", title="Bending moment magnitude at gp1")
lines!(ax1, 1:length(Mgp1_magnitude), Mgp1_magnitude; linewidth=2, label="gp1")
scatter!(ax1, 1:length(Mgp1_magnitude), Mgp1_magnitude; markersize=5)


# Subplot 2: Compare magnitudes across Gauss points
Mgp2_magnitude = [norm(m) for m in Mgp2_]
Mgp3_magnitude = [norm(m) for m in Mgp3_]
Mgp4_magnitude = [norm(m) for m in Mgp4_]

ax2 = Axis(fig[1,2], xlabel="Element index", ylabel="Magnitude |mᵢ|", title="Bending moment magnitude - all Gauss points")
lines!(ax2, 1:length(Mgp1_magnitude), Mgp1_magnitude; label="gp1", linewidth=2)
lines!(ax2, 1:length(Mgp2_magnitude), Mgp2_magnitude; label="gp2", linewidth=2)
lines!(ax2, 1:length(Mgp3_magnitude), Mgp3_magnitude; label="gp3", linewidth=2)
lines!(ax2, 1:length(Mgp4_magnitude), Mgp4_magnitude; label="gp4", linewidth=2)


# Subplot 3: Individual components of moment at gp1 (z-component most relevant for bending)
Mgp1_z = [m[3] for m in Mgp1_]  # Extract z-component (bending about z-axis)

ax3 = Axis(fig[2,1], xlabel="Element index", ylabel="mᵢ,z component", title="Bending moment z-component at gp1")
lines!(ax3, 1:length(Mgp1_z), Mgp1_z; linewidth=2, color=:red)
scatter!(ax3, 1:length(Mgp1_z), Mgp1_z; markersize=5, color=:red)


# Subplot 4: All three components at gp1
Mgp1_x = [m[1] for m in Mgp1_]
Mgp1_y = [m[2] for m in Mgp1_]

ax4 = Axis(fig[2,2], xlabel="Element index", ylabel="Component value", title="All moment components at gp1")
lines!(ax4, 1:length(Mgp1_x), Mgp1_x; label="mᵢ,x", linewidth=2)
lines!(ax4, 1:length(Mgp1_y), Mgp1_y; label="mᵢ,y", linewidth=2)
lines!(ax4, 1:length(Mgp1_z), Mgp1_z; label="mᵢ,z", linewidth=2)


fig
